In [290]:
import mss
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pyautogui
import time

In [291]:
# Variables

mon = 2 # set the screen capture on second monitor
templates = "./image_templates"
SCT = mss.mss()

#mon1_offset = [3439, 118] #this is the monitor offset resolution
mon1_offset = [3439, 118] #this is the monitor offset resolution


In [292]:
[ print(f"Monitor {x}\n" + str(SCT.monitors[x])) for x in [0, 1, 2] ]

Monitor 0
{'left': -3440, 'top': -118, 'width': 5360, 'height': 1440}
Monitor 1
{'left': 0, 'top': 0, 'width': 1920, 'height': 1080}
Monitor 2
{'left': -3440, 'top': -118, 'width': 3440, 'height': 1440}


[None, None, None]

In [293]:
#get_result('image_templates/button_adventure.png')

# template = cv2.imread('image_templates/button_adventure.png', cv2.IMREAD_GRAYSCALE)
# w, h = template.shape[::-1]

In [294]:
# Functions

def main_screen(height=1720):
    scr = SCT.grab({
                'left': SCT.monitors[2]['left'],
                'top': SCT.monitors[2]['top'],
                'width': 1720,
                'height': SCT.monitors[2]['height']
            })
    img = np.array(scr)
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def get_result(template_source: str) -> tuple:
    template = cv2.imread(template_source, cv2.IMREAD_GRAYSCALE)
    
    w, h = template.shape[::-1]

    screenshot = main_screen()
    result_matching = cv2.matchTemplate(screenshot,  template, cv2.TM_CCOEFF_NORMED)

    return cv2.minMaxLoc(result_matching)

def plot_match(template_source: str):
    template = cv2.imread(template_source, cv2.IMREAD_GRAYSCALE)
    
    w, h = template.shape[::-1]

    screenshot = main_screen()
    result_matching = cv2.matchTemplate(screenshot,  template, cv2.TM_CCOEFF_NORMED)

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result_matching)
    plt.figure(figsize=(16,16))
    plt.subplot(2, 1, 1)
    plt.imshow(screenshot, cmap="gray")
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(screenshot ,top_left, bottom_right, 255, 2)
    plt.subplot(2, 1, 2)
    plt.imshow(screenshot, cmap="gray")

def wait_for_template(template_source: str):
    min_val, max_val, min_loc, max_loc = get_result(template_source)
    retry_count = 0

    while(max_val < 0.8 and retry_count < 12):
        sleep(5)
        min_val, max_val, min_loc, max_loc = get_result(template_source)
        retry_count += 1
        print(f"{template_source} is not found yet. Confidence -> {max_val}")
        
    if retry_count == 12:
        raise NameError("Template matching not found")

def click_button(template: np.array, x_offset: int = 0, y_offset: int = 0, confidence: float = 0.8):
    a = cv2.imread(template, cv2.IMREAD_GRAYSCALE)
    w, h = a.shape

    min_val, max_val, min_loc, max_loc = get_result(template)
    if  max_val > 0.8:
        x_click = (max_loc[0] + w/2) - mon1_offset[0] + x_offset
        y_click = (max_loc[1] + h/2) - mon1_offset[1] + y_offset
        pyautogui.moveTo(x_click, y_click)
        pyautogui.click()
    else:
        print(f"Confidence not high enough: {max_val} < {confidence}")

def sleep(sec: int = 2):
    time.sleep(sec)


In [295]:
# pyautogui.position()

In [296]:
#DEBUG
#get_result("image_templates/label_one.png")
#plot_match("image_templates/label_one.png")
# click_button("image_templates/label_one.png")
# pyautogui.position()

#cv2.imread("image_templates/label_one.png", cv2.IMREAD_GRAYSCALE).shape

In [297]:
click_button("image_templates/button_adventure.png")
sleep()

In [298]:
click_button("image_templates/button_fight1_v-weak.png")#, y_offset=-30)
sleep()

In [299]:
click_button("image_templates/label_multiple_x.png", x_offset=150, y_offset=-50) #dont know why this works
sleep()

In [300]:
click_button("image_templates/label_one.png", y_offset=-20) #dont know why this works
sleep()

In [301]:
click_button("image_templates/button_fight_one_third.png")
sleep()

In [302]:
click_button("image_templates/button_fight2.png", y_offset=-30)
sleep(5)

In [303]:
wait_for_template("image_templates/button_confirm.png")
click_button("image_templates/button_confirm.png", y_offset=-30)
sleep()

In [304]:
wait_for_template("image_templates/button_attack.png")
click_button("image_templates/button_attack.png", y_offset=-30)
sleep(10)

image_templates/button_attack.png is not found yet. Confidence -> 0.582011342048645
image_templates/button_attack.png is not found yet. Confidence -> 0.582010805606842
image_templates/button_attack.png is not found yet. Confidence -> 0.9981313943862915


In [305]:
click_button("image_templates/button_back.png", y_offset=-30)
sleep(5)